In [26]:
# filter out sensitive info e.g

about = '''
My name is Steve Alila. 
I love building algorithms and extracting insights from data. 
For work, mainly I do web scraping while by-passing bots. 
I dream of traveling the world but I have never got time to do so. 
In this application I will log in with email "thecodeyai@mail.net" and password "Codingfor@Amen!thecodeyai"
'''

In [27]:
from langchain.chat_models import init_chat_model

model = init_chat_model("google_genai:gemini-2.5-flash-lite")
model.invoke("Hey")

AIMessage(content='Hey there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--748e83cf-64af-45b1-95a6-6fb9403b2c2b-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [28]:
from pydantic import BaseModel
from typing import List

class Person(BaseModel):
    name: str 
    likes: List[str]
    work: str 
    email: str
    password: str

In [36]:
from langchain.agents.structured_output import ToolStrategy
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware

agent = create_agent(
    model=model,
    response_format=ToolStrategy(Person),
    middleware=[
        PIIMiddleware("email", strategy="redact", apply_to_output=True),
        PIIMiddleware(
            "password",
            detector=r"(?i)(?:password|passwd|pwd)[\s:=]+[^\s]+",
            strategy="hash",
            apply_to_input=True,
            apply_to_output=True,
        ),
    ]
)

In [37]:
from langchain.messages import HumanMessage

response = agent.invoke({
    "messages": [HumanMessage(content=about)]
})

In [38]:
for res in response["structured_response"]:
    print(res)

('name', 'Steve Alila')
('likes', ['building algorithms', 'extracting insights from data', 'traveling the world'])
('work', 'web scraping while by-passing bots')
('email', '[REDACTED_EMAIL]')
('password', '<password_hash:b9ad5220>')
